<b>1. Import libraries</b>

In [1]:
from pytrends.request import TrendReq

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

<b>2. Initialize</b>

In [2]:
pytrends = TrendReq()

<b>3. General settings</b>

In [3]:
keywords = ['python']
cat = '0'
geo = 'ID'
gprop = 'youtube'

timeframes = [
    'today 5-y',
    'today 12-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

<b>4. Timeseries trends data</b>

In [4]:
pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=timeframes[4])

timeseries = pytrends.interest_over_time()
timeseries.reset_index(inplace=True)
timeseries.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
timeseries.head()

datetime  value  partial
0 2022-03-01 22:08:00      0    False
1 2022-03-01 22:16:00     28    False
2 2022-03-01 22:24:00      0    False
3 2022-03-01 22:32:00     23    False
4 2022-03-01 22:40:00      0    False

<b>5. Visualization</b>

In [5]:
from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import Label, ColumnDataSource, Select, DatetimeTickFormatter, RangeTool, NumeralTickFormatter, HoverTool
from bokeh.models.tools import BoxZoomTool, ResetTool

deftimeframe = timeframes[5]

pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=deftimeframe)

def get_data():
    ts = pytrends.interest_over_time()
    ts.reset_index(inplace=True)
    ts.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
    return dict(x=ts.datetime, y=ts.value)

def modify_doc(doc):
    source = ColumnDataSource(data=get_data())
    
    p = figure(
        tools = [BoxZoomTool(), ResetTool()],
        # background_fill_color="#efefef",
        toolbar_location="right",
        # x_axis_location="above",
        title=f"keyW: {keywords[0].title()} \n",
        width=850,
        height=350,
        x_axis_label="",
        y_axis_label=""
    )
    
    p.add_tools(HoverTool(
        tooltips=[
            ('datetime', '@x{%b %d, %Y}'),
            ('value', '@y{0.00 a}'),
        ],
        formatters={
            '@x': 'datetime', # use 'datetime' formatter for '@date' field
        },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
    ))
    
    p.line(x='x', y='y', source=source, color="#338CFF", line_width=3, alpha=0.8)
    p.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y"])
    
    # change some things about the x-axis
    p.xaxis.axis_line_width = 3
    p.xaxis.axis_line_color = "red"
    # p.xaxis.visible = False
    
    p.xgrid.visible = False
    
    # change some things about the y-axis
    p.yaxis.major_label_text_color = "green"

    select = Select(title="Timeframe", value=deftimeframe, options=timeframes, width=200)

    def update_data(attrname, old, new):
        try:
            if old != new:
                pytrends.build_payload(keywords, geo='ID', gprop='youtube', timeframe=select.value)
                source.data = get_data()
        except Exception as e:
            print(e)

    select.on_change('value', update_data)

    layout1 = column(row(select, p))

    doc.add_root(layout1)

show(modify_doc)